## Data

### Authors
- Brett Kinsella - Data Pipeline & Cleaning, Exploratory Data Analysis 


### Data Overview
- Dataset Name:
- Link to dataset:
- Number of observations:
- Number of variables:
- Description of variables:
- Descriptions of any shortcomings:

In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
# Credit for get_data module development goes to the teaching staff of COGS 108
# Setup code -- this only needs to be run once after cloning the repo!
# this code downloads the data from its source to the `data/00-raw/` directory

# if you don't already have these packages, uncomment this line
#%pip install requests tqdm

import sys
sys.path.append('./modules') # this tells python where to look for modules to import

import get_data # this is where we get the function we need to download data

datafiles = [
    { 'url': 'https://docs.google.com/spreadsheets/d/e/2PACX-1vRfclv62X33OubBIXBp8V4xIP-16WQiUA050EGYacqOG1N5vqn5UuyQJBBCBgIw6NvJ3VXRurHfOqoO/pub?gid=615866168&single=true&output=csv', 'filename':'pd_calls_for_service_2025_datasd.csv'},
]

get_data.get_raw(datafiles,destination_directory='data/00-raw/')


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: C:\Users\brett\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


Overall Download Progress: 100%|██████████| 1/1 [00:57<00:00, 57.19s/it]

Successfully downloaded: pd_calls_for_service_2025_datasd.csv


### Dataset: Police Calls for Service 2025
- Dataset Description:
    - This is 
- Major Concerns and Biases:
    - This is


In [ ]:
# if you don't already have these packages, uncomment this line
#%pip install pandas
#%pip install seaborn

import pandas as pd
import seaborn as sns

df = pd.read_csv('data/00-raw/pd_calls_for_service_2025_datasd.csv')
df.head()

,INCIDENT_NUM,DATE_TIME,DAY_OF_WEEK,ADDRESS_NUMBER_PRIMARY,ADDRESS_DIR_PRIMARY,ADDRESS_ROAD_PRIMARY,ADDRESS_SFX_PRIMARY,ADDRESS_DIR_INTERSECTING,ADDRESS_ROAD_INTERSECTING,ADDRESS_SFX_INTERSECTING,CALL_TYPE,DISPOSITION,BEAT,PRIORITY
0,E25120020309,2025-12-15 9:48:49,2,4200,NaN,03RD,AVE,NaN,NaN,NaN,487R,K,627,9
1,E25120025105,2025-12-18 16:32:13,5,5500,NaN,MILDRED,ST,NaN,NaN,NaN,10852R,K,622,9
2,E25120028508,2025-12-20 22:43:23,7,500,NaN,05TH,AVE,NaN,NaN,NaN,415,A,523,2
3,E25120028604,2025-12-21 0:02:05,1,0,NaN,SHOPS LOG,NaN,NaN,NaN,NaN,SHOPS,W,-1,4
4,E25120028605,2025-12-21 0:02:41,1,0,NaN,MCRT LOG,NaN,NaN,NaN,NaN,INFO,W,-1,2
